In [1]:
import gymnasium as gym
import os

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from envs.curriculum_halfcheetah import HalfCheetahEnv

from policy.curriculum_ppo import CurriculumPPO

2023-11-02 15:25:57.779646: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 15:25:57.779687: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 15:25:57.779722: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 15:25:57.792812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def make_env(env_id: str, rank: int, task = None, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        check_env(env) # check the environment
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [3]:
env_id = "HalfCheetah-v4"
num_cpu = 4

In [4]:
from gymnasium.envs.registration import register

register(
     id="Curriculum/HalfCheetah-v4",
     entry_point="envs.curriculum_halfcheetah:HalfCheetahEnv",
     max_episode_steps=1000,
)

/home/kh-ryu/anaconda3/envs/curriculum/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment Curriculum/HalfCheetah-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [4]:
training_env = SubprocVecEnv([make_env(env_id, i, task=None) for i in range(num_cpu)])

2023-11-02 15:26:21.253376: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 15:26:21.253375: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 15:26:21.253412: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 15:26:21.253414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 15:26:21.253433: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register fa

In [5]:
obs = training_env.reset()

In [6]:
obs

array([[-0.01546256, -0.09433607, -0.07514334,  0.03412488,  0.0294379 ,
         0.02307702, -0.02326449,  0.09944199, -0.09217254, -0.04577258,
         0.02201951, -0.10096182, -0.02091756, -0.0159225 ,  0.05408456,
         0.02146591,  0.03553727],
       [-0.04753733,  0.05007293, -0.04391825, -0.00296181,  0.09614744,
         0.09233144,  0.04495799,  0.00824537, -0.04221904,  0.0213643 ,
         0.02173219,  0.21178388, -0.11120208, -0.0377605 ,  0.20427716,
         0.0646703 ,  0.06630634],
       [-0.06254949, -0.03080787,  0.00221319,  0.07824188,  0.05511279,
        -0.03637068,  0.08484338, -0.00581802, -0.16384425, -0.17294115,
        -0.15048314,  0.08414589,  0.01287157,  0.10783424,  0.07224309,
         0.02105718,  0.02840381],
       [-0.04145585, -0.09970198,  0.09469205, -0.04031976, -0.0372028 ,
         0.07834221,  0.01703259, -0.00573807, -0.02696203, -0.02435587,
         0.10023136, -0.08864599, -0.02917202,  0.0882539 ,  0.058035  ,
         0.00915167

In [ ]:
model = CurriculumPPO("MlpPolicy", training_env, task_list=None, task_threshold=None, verbose=1, tensorboard_log="./tensorboard/")

In [ ]:
for _ in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = training_env.step(action)
    print(dones)
    print(_)
    if dones.any():
        break